In [42]:
import pickle
import pandas as pd
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
path_models = "C:/Users/asus-pc/Documents/PBA/Tugas Akhir/Untitled Folder/04. Model Training/Models/"

path_rfc = path_models + 'best_rfc.pickle'
with open(path_rfc, 'rb') as  data:
    rfc_model = pickle.load(data)

In [44]:
path_tfidf = "C:/Users/asus-pc/Documents/PBA/Tugas Akhir/03. Feature Engineering/Pickles_title/tfidf.pickle"
with open(path_tfidf, 'rb') as data:
    tfidf = pickle.load(data)

D:\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [45]:
kode_label = {
    'notification of information': 0,
    'donation': 1,
    'criticisms': 2,
    'hoax': 3,
}

In [46]:
def membuat_fitur(text):
    
    # Dataframe creation
    lemmatized_text_list = []
    df = pd.DataFrame(columns=['title'])
    df.loc[0] = text
    df['title_parsed_1'] = df['title'].str.replace("\r", " ")
    df['title_parsed_1'] = df['title_parsed_1'].str.replace("\n", " ")
    df['title_parsed_1'] = df['title_parsed_1'].str.replace("    ", " ")
    df['title_parsed_1'] = df['title_parsed_1'].str.replace('"', '')
    df['title_parsed_2'] = df['title_parsed_1'].str.lower()
    df['title_parsed_3'] = df['title_parsed_2']
    
    punctuation_signs = list("?:!.,;")
    for punct_sign in punctuation_signs:
        df['title_parsed_3'] = df['title_parsed_3'].str.replace(punct_sign, '')
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = df.loc[0]['title_parsed_3']  
    sentence = text
    lemmatized_text = stemmer.stem(sentence)
    lemmatized_text_list.append(lemmatized_text)
    df['title_parsed_4'] = lemmatized_text_list
    df['title_parsed_5'] = df['title_parsed_4']
    
    punctuation_signs = list("?:!.,;")
    STOPWORDS= stopwords.words('Indonesian')
    STOPWORDS.extend(['covid','covid-19','covid-19,','korona','2020','corona', 'corona,','2021','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','ribu','juta','-'])
    stop_words = list(STOPWORDS)
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['title_parsed_5'] = df['title_parsed_5'].str.replace(regex_stopword, '')
    df['title_parsed_5'] = df['title_parsed_5'].str.replace('-', '')
    df = df.rename(columns={'title_parsed_5': 'title_parsed'})
    df = df['title_parsed']
    #df = df.rename(columns={'title_parsed_6': 'title_parsed'})

    # TF-IDF
    features = tfidf.transform(df).toarray()
    
    return features

In [47]:
def get_nama_label(label_id):
    for label, id_ in kode_label.items():    
        if id_ == label_id:
            return label

In [48]:
def prediksi_dari_teks(teks):
    
    # Predict using the input model
    prediksi_rfc = rfc_model.predict(membuat_fitur(teks))[0]
    prediksi_rfc_proba = rfc_model.predict_proba(membuat_fitur(teks))[0]
    
    # Return result
    label_rfc = get_nama_label(prediksi_rfc)
    
    print("Prediksi menggunakan RFC model adalah %s." %(label_rfc) )
    print("Kondisional Probability: %a" %(prediksi_rfc_proba.max()*100))
    return label_rfc,prediksi_rfc_proba.max()*100

In [49]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="tugasakhir"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT title FROM news where label is null or label=''")
data_berita = mycursor.fetchall()


In [50]:
for baris in data_berita:
    print(baris[0])
    nama_label,label_persen=prediksi_dari_teks(baris[0])
    if(label_persen>=65):
        if("'" not in str(baris[0])):
            sql = "UPDATE news SET label = '"+nama_label+ "' WHERE title = '"+str(baris[0])+"'"    
            print(sql)
            mycursor.execute(sql)
            mydb.commit()
            
        else:
            sql = 'UPDATE news SET label = "'+nama_label+ '" WHERE title = "'+str(baris[0])+'"'
            print(sql)
            mycursor.execute(sql)
            mydb.commit()
    else:
        if("'" not in str(baris[0])):
            sql = "UPDATE news SET label = '"+"other"+ "' WHERE title = '"+str(baris[0])+"'"    
            print(sql)
            mycursor.execute(sql)
            mydb.commit()
            
        else:
            sql = 'UPDATE news SET label = "'+'other'+ '" WHERE title = "'+str(baris[0])+'"'
            print(sql)
            mycursor.execute(sql)
            mydb.commit()

In [51]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="tugasakhir"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT count(*) FROM news where label is null or label=''")
myresult = mycursor.fetchall()
print(myresult)

[(0,)]


NameError: name 'df_misclassified' is not defined

In [57]:
output_article("UPDATE Corona 31 Maret di 32 Provinsi: Kasus B.")

TypeError: string indices must be integers